In [1]:
import numpy as np, pandas as pd
from sklearn.compose import ColumnTransformer
# 여러 열에 대해 서로 다른 전처리기를 적용할 수 있게 함 
# 숫자형 열에는 표준화 _ StandardScaler
# 범주형 열에는 _ OneHotEncoder 
# 이런 걸 한 번에 처리할 수 있음 
from sklearn.pipeline import Pipeline
# 여러 단계의 머신러닝 작업을 하나로 묶어서 깔끔 처리하게 해주는 도구 
# pipe = Pipeline([
#     ('imputer', SimpleImputer(strategy='mean')),      # 결측값 처리
#     ('scaler', StandardScaler()),                     # 정규화
#     ('classifier', LogisticRegression())              # 모델 학습
# ])
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
# 결측값 처리 위한 도구 불러오는 코드 
# 데이터셋에 **NaN(결측값)**이 있을 때, 그 자리를 평균, 중앙값, 최빈값 등으로 채워주는 역할
# - strategy='mean': 평균값으로 채움
# - strategy='median': 중앙값으로 채움
# - strategy='most_frequent': 가장 많이 나온 값으로 채움
# - strategy='constant': 사용자가 지정한 값으로 채움

In [2]:
import matplotlib.pyplot as plt
from matplotlib import rc
# rc: runtime configuration
rc("font", family='Malgun Gothic')
plt.rcParams["axes.unicode_minus"] = False

In [3]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
submission = pd.read_csv("./sample_submission.csv")

In [6]:
train.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,0,42,technician,married,secondary,no,7,no,no,cellular,25,aug,117,3,-1,0,unknown,0
1,1,38,blue-collar,married,secondary,no,514,no,no,unknown,18,jun,185,1,-1,0,unknown,0
2,2,36,blue-collar,married,secondary,no,602,yes,no,unknown,14,may,111,2,-1,0,unknown,0
3,3,27,student,single,secondary,no,34,yes,no,unknown,28,may,10,2,-1,0,unknown,0
4,4,26,technician,married,secondary,no,889,yes,no,cellular,3,feb,902,1,-1,0,unknown,1


In [7]:
test.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,750000,32,blue-collar,married,secondary,no,1397,yes,no,unknown,21,may,224,1,-1,0,unknown
1,750001,44,management,married,tertiary,no,23,yes,no,cellular,3,apr,586,2,-1,0,unknown
2,750002,36,self-employed,married,primary,no,46,yes,yes,cellular,13,may,111,2,-1,0,unknown
3,750003,58,blue-collar,married,secondary,no,-1380,yes,yes,unknown,29,may,125,1,-1,0,unknown
4,750004,28,technician,single,secondary,no,1950,yes,no,cellular,22,jul,181,1,-1,0,unknown


In [4]:
unique_in_train = set(train.columns) - set(test.columns)
unique_in_train

{'y'}

# 입력 변수(X), 타겟(y) 분리

In [5]:
X = train.drop(columns=["y"])
y = train["y"]

X_test = test.copy()

# 전처리

In [6]:
# one hto encoding / scailing은 train + test 합쳐서 fit 하고 다시 나눠주는 게 안전 
full = pd.concat([X, X_test], axis=0)

In [16]:
X.columns

Index(['id', 'age', 'job', 'marital', 'education', 'default', 'balance',
       'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign',
       'pdays', 'previous', 'poutcome'],
      dtype='object')

In [19]:
X.dtypes

id            int64
age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
dtype: object

In [20]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 17 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         750000 non-null  int64 
 1   age        750000 non-null  int64 
 2   job        750000 non-null  object
 3   marital    750000 non-null  object
 4   education  750000 non-null  object
 5   default    750000 non-null  object
 6   balance    750000 non-null  int64 
 7   housing    750000 non-null  object
 8   loan       750000 non-null  object
 9   contact    750000 non-null  object
 10  day        750000 non-null  int64 
 11  month      750000 non-null  object
 12  duration   750000 non-null  int64 
 13  campaign   750000 non-null  int64 
 14  pdays      750000 non-null  int64 
 15  previous   750000 non-null  int64 
 16  poutcome   750000 non-null  object
dtypes: int64(8), object(9)
memory usage: 97.3+ MB


In [7]:
num_cols = full.select_dtypes

In [8]:
cat_cols = full.select_dtypes(include=["object"]).columns.tolist()
# OneHotEncoding
full = pd.get_dummies(full, columns=cat_cols, drop_first=True)
# Scailing 
# scaler = StandardScaler()
# full[num_cols] = scaler.fit_transform(full[num_cols])

In [22]:
cat_cols

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'poutcome']

In [9]:
X = full.iloc[:len(X), :]
X_test = full.iloc[len(X):, :]

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
# stratify=y: 데이터 나눌 때 비율 유지하겠다는 뜻 _ 각 클래스(0,1)의 비율이 전체 데이터의 y와 같게 나누어짐 

In [11]:
X_train = X_train.replace("unknown", np.nan)
X_valu = X_val.replace("unknown", np.nan)

num_cols = X_train.select_dtypes(include=["number"]).columns.tolist()
cat_cols = [c for c in X_train.columns if c not in num_cols]

num_pipe = Pipeline([("imp", SimpleImputer(strategy="median")),
                    ("sc", StandardScaler())])
# pipeline 안에 여러 단계가 있을 때, 각각을 식별하는 용도로 imp, sc를 씀 
# 예를 들어 GridSearchCV를 사용할 때, 각 단계의 파라미터를 조정하려면 이름이 있어야 함 
# param_grid = {
#     'imp__strategy': ['mean', 'median'],
#     'sc__with_mean': [True, False]
# }
cat_pipe = Pipeline([("imp", SimpleImputer(strategy="most_frequent")),
                     ("oh", OneHotEncoder(handle_unknown="ignore"))])

pre = ColumnTransformer([
    ("num", num_pipe, num_cols),
    ("cat", cat_pipe, cat_cols),
])

# ML Model

## Sklearn

## kernel
- 복잡한 패턴을 잡으려면 입력을 고차원 특징공간으로 바꿔서 선형으로 자르면 쉽다(“특징 맵” φ(x)).
- 그런데 φ(x)가 엄청 고차원이면 직접 계산이 너무 비싸다.
- 커널 트릭: 두 점의 고차원 내적 ⟨φ(x), φ(z)⟩을 커널 함수 K(x, z)로 바로 계산한다.
- 즉, 맵핑은 “암시적으로” 하고, 우리는 K만 계산하면 된다.
- SVM/커널릿들이 결국 “커널 행렬(Gram matrix)” K_ij = K(x_i, x_j)만으로 학습한다.
### RBF(가우시안) 커널
정의: K(x,z)=exp(−γ∥x−z∥2)\
또는 γ=1/2σ^2로 쓰기도 함.\
해석: 두 샘플이 가까우면(유클리드 거리 작으면) 유사도≈1, 멀면 0에 수렴.\
즉, **“가까운 이웃은 비슷하다”**는 가정으로 매끈한 곡면 결정경계를 만든다.\
특징: 무한차원 특징공간에 해당(이론적으로 매우 표현력 풍부). 그래서 범용으로 잘 먹힌다.

**핵심 하이퍼파라미터**
- γ (gamma): “가까움”의 기준(스케일).
    - 작은 γ → 넓은 종(bandwidth 큼) → 부드러운 경계(바이어스↑, 분산↓).\
    - 큰 γ → 좁은 종(bandwidth 작음) → 요철 많은 경계(바이어스↓, 분산↑, 과적합 위험).\
    - 관계: σ=1/2γ\
    - scikit-learn 기본값 gamma="scale"은 \gamma = \frac{1}{\text{n_features} \cdot \text{Var}(X)}.
- C(규제 강도): 마진 최대화 vs 오분류 패널티의 트레이드오프.
    - 큰 C: 오분류를 덜 허용 → 경계가 복잡해질 수 있음(과적합 위험).
    - 작은 C: 오분류 좀 허용 → 경계가 부드러움(과소적합 위험).
- C와 γ의 상호작용:
    - 둘 다 복잡도를 키우는 방향으로 작동 가능 → **(C↑, γ↑)**는 가장 공격적(과적합 주의).
    - 보통 로그스케일 그리드 서치로 함께 튜닝한다.

**꼭 지켜야 하는 전처리**
- 스케일링 필수(표준화/MinMax). 거리 기반이므로 단위·스케일이 다르면 왜곡됨.
- 원-핫 고차원 희소벡터가 너무 많으면 거리의 의미가 약해질 수 있음 → 트리계열이나 선형 모델 검토.

**언제 RBF가 좋은가?**
- 중/소규모 표본에서 비선형 경계가 필요한 분류/회귀.
- 특징 수는 적당하고(수~수십), 스케일링이 잘 되어 있을 때.

**한계와 대안**
- 시간/메모리 복잡도가 샘플 수에 비선형으로 커진다(커널 행렬 O(n²)).\
데이터가 아주 크면 Linear SVM, XGBoost/LightGBM, 혹은 근사 커널(Nystrom, RFF) 고려.
- 확률 출력이 필요하면 probability=True로 Platt scaling(추가 비용).

# 🌲 Step-by-Step: 랜덤 포레스트의 작동 방식

1. 부트스트랩 샘플링 (Bagging)
- 전체 이메일 데이터에서 중복 허용하며 랜덤하게 샘플을 뽑아 여러 개의 작은 데이터셋을 만든다.
- 예를 들어, 400개의 트리를 만들기로 했으니, 400개의 서로 다른 샘플링된 데이터셋이 생긴다.
  
2. 랜덤 피처 선택
- 각 트리는 학습할 때 전체 피처 중 일부만 랜덤하게 선택해서 사용한다.
- 예를 들어, 어떤 트리는 "무료"와 "첨부파일"만 보고 판단하고, 다른 트리는 "발신자"와 "이메일 길이"만 본다.

3. 개별 결정 트리 학습
- 각 트리는 자신만의 데이터와 피처를 가지고 스팸인지 아닌지를 판단하는 규칙을 만든다.
- 어떤 트리는 "무료"라는 단어가 있으면 스팸이라고 하고, 다른 트리는 "첨부파일"이 있으면 스팸이라고 할 수도 있어.
  
4. 예측 시 투표
- 새로운 이메일이 들어오면, 400개의 트리 각각이 자신의 기준으로 판단한다.
- 예: 400개 중 320개가 "스팸"이라고 하면 → 최종 예측은 "스팸"

# 🌱 GBDT의 작동 원리: “잔차를 줄여가며 트리를 쌓는다”

1. 첫 번째 트리
- 처음엔 아주 단순한 트리를 하나 만든다.
- 이 트리는 대충 평균값이나 간단한 규칙으로 예측을 한다.
- 당연히 오차(잔차)가 많이 생기겠지?
  
2. 두 번째 트리
- 이제 첫 번째 트리가 틀린 부분(잔차)을 학습해서 보정하는 트리를 만든다.
- 예를 들어, 첫 번째 트리가 5억이라고 예측했는데 실제는 6억이면, 두 번째 트리는 그 1억 차이를 줄이려고 노력함.

3. 세 번째, 네 번째...
- 이런 식으로 잔차를 줄이는 트리들을 계속 쌓아가면서, 점점 더 정교한 예측을 하게 돼.
- 각 트리는 약한 모델이지만, 모두 합치면 강력한 모델이 되는 거야.


---

## 🚀 XGBClassifier란?

`XGBClassifier`는 **XGBoost (Extreme Gradient Boosting)** 라이브러리의 분류기 버전이야. 이름부터 “익스트림”이 붙었지? 그만큼 성능과 효율을 극한까지 끌어올린 GBDT의 업그레이드 버전이라고 보면 돼.

---

## 🔧 GBDT vs XGBoost: 뭐가 달라졌을까?

| 항목 | GradientBoostingClassifier | XGBClassifier |
|------|-----------------------------|---------------|
| **학습 방식** | 잔차 기반 부스팅 | 잔차 + 정규화 + 더 똑똑한 트리 |
| **속도** | 느림 (순차적 학습) | 빠름 (병렬 처리, 캐시 최적화) |
| **과적합 방지** | learning_rate로 조절 | 정규화(L1/L2), early stopping 등 다양함 |
| **결측치 처리** | 직접 처리 필요 | 자동 처리 가능 |
| **성능** | 좋음 | 더 좋음 (특히 대규모 데이터에서) |
| **사용 편의성** | 비교적 단순 | 파라미터 많지만 튜닝 여지 큼 |

---

## 🧠 XGBClassifier의 핵심 발전 포인트

### 1. **정규화(Regularization)**
- GBDT는 트리를 계속 쌓다 보면 과적합 위험이 커.
- XGBoost는 **L1, L2 정규화**를 통해 모델 복잡도를 제어해서 과적합을 방지해.

### 2. **병렬 처리**
- GBDT는 트리를 하나씩 순차적으로 학습하지만,
- XGBoost는 **노드 분할을 병렬로 처리**해서 훨씬 빠르게 학습함.

### 3. **결측치 자동 처리**
- XGBoost는 결측값이 있어도 알아서 처리해줘서 전처리 부담이 줄어들어.

### 4. **Early Stopping**
- 검증 성능이 더 이상 좋아지지 않으면 학습을 멈추는 기능.
- 불필요한 트리 생성을 막고, 과적합도 줄여줌.

### 5. **트리 구조 최적화**
- 기존 GBDT는 깊이 우선 방식으로 트리를 만들지만,
- XGBoost는 **최적 분할을 더 똑똑하게 계산**해서 성능이 더 좋아.

---

## 🏡 예시로 다시 돌아가보자: 집값 예측

- GBDT는 집값을 예측할 때, 방 개수나 평수 같은 피처를 기반으로 잔차를 줄여가며 트리를 쌓아.
- XGBoost는 같은 작업을 하되, **더 빠르게**, **더 정교하게**, 그리고 **과적합을 막으면서** 예측해.

---

## 🎓 한 줄 요약

> XGBClassifier는 GBDT를 “속도, 성능, 안정성” 면에서 극한까지 끌어올린 진화형 모델이야. 실무에서 많이 쓰이는 이유가 다 있어!

---

---

## 🌱 LGBMClassifier란?

**LightGBM**은 Microsoft에서 만든 GBDT 기반의 머신러닝 라이브러리야.  
핵심 목표는 **속도와 메모리 효율을 극대화하면서도 성능은 유지하거나 더 좋게** 만드는 것!

---

## 🧠 GBDT → XGBoost → LightGBM: 진화 흐름

| 모델 | 특징 |
|------|------|
| GBDT | 기본 부스팅. 느리고 과적합 위험 있음 |
| XGBoost | 빠르고 정교함. 정규화, 병렬 처리 도입 |
| **LightGBM** | 훨씬 빠름. 대용량 데이터에 최적화. 희소/고차원 데이터도 잘 처리 |

---

## 🔍 LightGBM의 핵심 기술

### 1. **Leaf-wise 성장 방식**
- 기존 GBDT나 XGBoost는 트리를 **depth-wise**로 키워 (균형 있게).
- LightGBM은 **leaf-wise**로 키워서 **오차를 가장 많이 줄이는 방향으로만 성장**함.
- 결과적으로 **더 깊고 정교한 트리**가 만들어져서 성능이 좋아짐.

### 2. **Histogram 기반 학습**
- 연속형 피처를 **구간별로 묶어서(histogram)** 처리함.
- 덕분에 **메모리 사용량이 확 줄고**, **속도도 빨라짐**.

### 3. **희소 데이터 처리에 강함**
- 텍스트 벡터처럼 **0이 많은 희소한 데이터**도 효율적으로 처리함.
- XGBoost보다 이 부분에서 더 유리함.

### 4. **GPU 지원**
- GPU를 활용해서 대규모 데이터도 빠르게 학습 가능!

---

## 🏡 예시: 아파트 가격 예측

- GBDT는 잔차를 줄이는 트리를 쌓아가며 예측
- XGBoost는 병렬 처리와 정규화로 더 빠르고 안정적으로 예측
- **LightGBM은**:
  - 더 빠르게
  - 더 적은 메모리로
  - 더 정교한 트리를 만들어서
  - **대규모 데이터에서도 훌륭한 성능**을 보여줌

---

## ⚙️ 주요 하이퍼파라미터 (기본값 기준)

| 파라미터 | 설명 |
|----------|------|
| `num_leaves=31` | 트리에서 사용할 리프 노드 수. 많을수록 복잡한 모델 가능 |
| `max_depth=-1` | 트리 깊이 제한 없음. 대신 num_leaves로 제어 |
| `learning_rate=0.1` | 학습 속도. 작을수록 천천히, 과적합 방지 |
| `n_estimators=100` | 트리 개수. 많을수록 정교하지만 느려짐 |

---

## ✅ 장점 요약

- **속도 빠름**: 특히 대용량 데이터에서 압도적
- **메모리 효율 좋음**
- **희소/고차원 데이터에 강함**
- **성능 우수**: 특히 표형(tabular) 데이터에서 강력

## ❌ 단점 요약

- **leaf-wise 방식은 과적합 위험 있음** → 적절한 튜닝 필요
- **파라미터 많음** → 초보자에겐 진입장벽

---

## 🎓 한 줄 요약

> LightGBM은 “빠르고 똑똑한 GBDT”로, 대규모 데이터와 복잡한 문제를 효율적으로 해결하는 머신러닝의 실무 최강자야!

---


| 모델         | 원리             | 장점                 | 단점                | 잘 쓰이는 곳               |
| ---------- | -------------- | ------------------ | ----------------- | --------------------- |
| **LogReg** | 선형 + 시그모이드     | 해석력, 빠름, 확률 잘 보정   | 비선형 못 잡음          | 텍스트, 해석 필요한 의료/사회 데이터 |
| **SVM**    | 마진 최대화 + 커널    | 복잡한 경계 가능, 소규모에 강함 | 대규모 느림, 확률 보정 필요  | 중/소규모 비선형 분류          |
| **RF**     | 배깅+랜덤트리        | 범용 강함, 전처리 자유로움    | 해석 어려움, 희소고차원 비효율 | 표형 데이터, 변수 중요도 분석     |
| **GBDT**   | 부스팅            | 비선형 잘 학습, 표형 강함    | 느림, 튜닝필요          | Kaggle baseline       |
| **XGB**    | GBDT+정규화+2차최적화 | 성능↑, 세밀튜닝          | 메모리↑, 복잡          | Kaggle SOTA           |
| **LGBM**   | 리프 기반 GBDT     | 속도↑, 메모리 효율↑       | 소규모 불안정           | 대규모 데이터, Kaggle SOTA  |


In [30]:
!pip install xgboost

In [32]:
!pip install lightgbm

In [11]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# 0) 'unknown'을 결측치 취급
X_train = X_train.replace("unknown", np.nan)
X_val   = X_val.replace("unknown", np.nan)

# 0.5) bool/boolean 컬럼을 object(문자열)로 바꾸거나, 0/1 숫자로 바꾼다 (둘 중 하나 택1)

# [옵션 A] 카테고리로 처리하고 싶다 → object로 캐스팅
bool_cols = X_train.select_dtypes(include=["bool", "boolean"]).columns.tolist()
if bool_cols:
    X_train[bool_cols] = X_train[bool_cols].astype("object")
    X_val[bool_cols]   = X_val[bool_cols].astype("object")

# [옵션 B] 숫자(0/1)로 처리하고 싶다 → int8로 캐스팅 (위 A 대신 이걸 쓰려면 A는 주석 처리)
# bool_cols = X_train.select_dtypes(include=["bool", "boolean"]).columns.tolist()
# if bool_cols:
#     X_train[bool_cols] = X_train[bool_cols].astype("int8")
#     X_val[bool_cols]   = X_val[bool_cols].astype("int8")

# 1) 컬럼 타입 분리 (캐스팅 후에 다시 뽑기 중요!)
num_cols  = X_train.select_dtypes(include=["int64", "float64", "int32", "float32", "int16", "float16", "int8"]).columns.tolist()
cat_cols  = X_train.select_dtypes(include=["object", "category"]).columns.tolist()

print("num:", num_cols)
print("cat:", cat_cols)

# 2) 파이프라인
num_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median")),
    ("sc",  StandardScaler()),
])
cat_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("oh",  OneHotEncoder(handle_unknown="ignore")),
])

pre = ColumnTransformer([
    ("num", num_pipe, num_cols),
    ("cat", cat_pipe, cat_cols),
])


num: ['id', 'age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
cat: ['job_blue-collar', 'job_entrepreneur', 'job_housemaid', 'job_management', 'job_retired', 'job_self-employed', 'job_services', 'job_student', 'job_technician', 'job_unemployed', 'job_unknown', 'marital_married', 'marital_single', 'education_secondary', 'education_tertiary', 'education_unknown', 'default_yes', 'housing_yes', 'loan_yes', 'contact_telephone', 'contact_unknown', 'month_aug', 'month_dec', 'month_feb', 'month_jan', 'month_jul', 'month_jun', 'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep', 'poutcome_other', 'poutcome_success', 'poutcome_unknown']


In [ ]:
from tqdm import tqdm
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report 

models = {
    "logreg": LogisticRegression(max_iter=2000, class_weight="balanced"),
    # 선형 모델 z = w*x + b를 sigmoid σ(z)로 변환해서 P(y=1|x) 확률을 출력 / 다중 분류는 softmax
    # max_iter=2000 → 수렴할 때까지 경사하강법 반복 횟수 충분히 늘림.
    # class_weight="balanced" → 클래스 비율이 불균형할 때 자동으로 가중치 조정.
    # 장점: 빠름, 해석력(가중치 방향/크기) 좋음, 확률 출력이 안정적.
    # 단점: 선형 경계만 학습 → 비선형 패턴은 잘 못 잡음. 스케일링 필수.
    "svm": SVC(kernel="rbf", probability=True, class_weight="balanced"),
    # 원리: “마진”을 최대화하는 결정경계를 찾음. 비선형 패턴은 커널(여기선 RBF)로 변환해 분리.
    # = 데이터를 가장 잘 분류하면서도 클래스 간의 여유 공간을 최대한 확보하는 경계선을 찾는다
    # kernel="rbf" → 가우시안(RBFRadial Basis Function) 커널로 복잡한 곡선형 경계 표현 가능.
    # probability=True → Platt scaling을 이용해 확률 값까지 출력 (추가 연산 있음).
    # class_weight="balanced" → 불균형 클래스 대응.
    # 장점: 중/소규모 데이터에서 비선형 분류 잘함. 마진 극대화라 일반화 성능이 좋음.
    # 단점: 데이터 크면 학습/예측 느려짐. 파라미터(C, gamma) 튜닝 필수.
    "rf": RandomForestClassifier(n_estimators=400, random_state=42, n_jobs=-1),
    # 원리: 배깅(Bagging) 기반 앙상블. 부트스트랩 샘플 + 랜덤 피처 선택으로 여러 트리 학습 → 투표.
    # n_estimators=400 → 트리 400개. 많을수록 안정성↑, 시간/메모리 비용도 ↑.
    # n_jobs=-1 → 멀티코어 병렬 처리.
    # 장점: 전처리 거의 필요 없음, 범용성 높음, 과적합에 강함.
    # 단점: 해석력 낮음, 매우 희소/고차원 데이터엔 비효율.
    "gbdt": GradientBoostingClassifier(random_state=42),
    # 원리: 부스팅(Boosting). 이전 트리의 오차(잔차)를 점점 줄여가며 약한 트리들을 순차적으로 쌓음.
    # 하이퍼파라미터 (기본값):
    # learning_rate=0.1 (작을수록 천천히, 과적합 방지)
    # n_estimators=100 (트리 개수, 많을수록 정교)
    # max_depth=3 (얕은 트리로 단계적 보정)
    # 장점: 복잡한 비선형 패턴을 잘 잡음. 표형(tabular)에서 매우 강력.
    # 단점: 파라미터 많음, 학습 느림, 과적합 주의 → 보통 XGBoost/LightGBM으로 대체.
    "xgb": XGBClassifier(
        n_estimators=600, max_depth=6, learning_rate=0.05,
        subsample=0.9, colsample_bytree=0.9, eval_metric="logloss", 
        tree_method="gpu_hist",
        predictor="gpu_predictor"
    ),
    # 원리: GBDT 발전형. 2차 도함수까지 사용한 정교한 최적화 + 정규화로 과적합 방지.
    # 주요 파라미터:
    # n_estimators=600 → 트리 수.
    # max_depth=6 → 개별 트리 최대 깊이. 깊을수록 복잡 패턴, 과적합 위험↑.
    # learning_rate=0.05 → 작은 학습률로 천천히 학습 (트리 개수는 늘려야 함).
    # subsample=0.9 → 샘플 일부만 사용해 과적합 방지.
    # colsample_bytree=0.9 → 피처 일부만 사용해 트리 학습.
    # tree_method="hist" → 히스토그램 기반 빠른 분할 (GPU 쓰려면 "gpu_hist").
    # 장점: 속도/성능 균형 최고, 파라미터 세밀 조정 가능.
    # 단점: 튜닝 필요, 메모리 사용량 ↑.
    "lgbm": LGBMClassifier(
        n_estimators=1000, learning_rate=0.05,
        subsample=0.9, colsample_bytree=0.9, objective="binary",
        device="gpu"
    )
    # (LightGBM)
    # 원리: 또 다른 GBDT 구현. 리프 중심(Tree leaf-wise) 성장 + 히스토그램 기반 분할로 속도와 메모리 효율 뛰어남.
    # 주요 파라미터:
    # n_estimators=1000 → 트리 개수.
    # learning_rate=0.05 → 낮은 학습률, 과적합 방지.
    # subsample=0.9, colsample_bytree=0.9 → 데이터/특징 서브샘플링.
    # objective="binary" → 이진 분류.
    # 장점: 대규모 데이터에서 빠르고 메모리 효율적. 카테고리형 직접 처리 가능(categorical_feature).
    # 단점: 작은 데이터에서 오히려 불안정할 수 있음. 카테고리 처리 잘못 쓰면 성능 저하.
}

print("\n=== ML 모델 비교 ===")
ml_results = []
for name, est in tqdm(models.items(), desc="모델 학습 진행중", total=len(models)):
    pipe = Pipeline([("pre", pre), ("est", est)])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_val)
    try: 
        y_prob = pipe.predict_proba(X_val)[:, 1]
    except Exception:
        y_prob = None
    
    acc = accuracy_score(y_val, y_pred)
    f1m = f1_score(y_val, y_pred, average="macro")
    auc = (roc_auc_score(y_val, y_prob) if y_prob is not None else np.nan)
    ml_results.append((name, acc, f1m, auc))
    print(f"{name:>6s}")
    # >: 오른쪽 정렬, 6: 총 너비 6칸, s: - 문자열(string) 타입 (참고: ^6s이면 가운데 정렬임)

best_name, *_ = sorted(ml_results, key=lambda t: (np.nan_to_num(t[3]), t[2]), reverse=True)[0]
# t[3]: AUC 점수, t[2]: F1 macro 점수 
# - AUC가 np.nan일 수도 있으니까, 그걸 0.0으로 바꿔서 비교 가능하게 만들어줌.
# - 즉, AUC가 없는 모델은 성능이 낮다고 간주함.
# - AUC를 우선 기준으로, F1 macro를 보조 기준으로 정렬함.
# [0]: - 정렬된 리스트에서 가장 성능이 좋은 모델 하나만 선택
# - 나머지 값들(acc, f1m, auc)은 _로 무시
print(f"\nBest ML model (AUC/F1 기준): {best_name}")


=== ML 모델 비교 ===


모델 학습 진행중:  17%|███████████                                                       | 1/6 [00:25<02:09, 25.81s/it]

logreg


In [12]:
import time
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

def is_binary(y):
    u = np.unique(y)
    return len(u) == 2 and set(u) <= {0, 1}

def get_scores_for_auc(pipe, X):
    # predict_proba 있으면 1클래스 확률, 없으면 decision_function 사용
    if hasattr(pipe, "predict_proba"):
        p = pipe.predict_proba(X)
        if p.ndim == 2 and p.shape[1] >= 2:
            return p[:, 1]
    if hasattr(pipe, "decision_function"):
        return pipe.decision_function(X)
    return None

def evaluate_model(estimator, name):
    pipe = Pipeline([("pre", pre), ("est", estimator)])
    t0 = time.time()
    pipe.fit(X_train, y_train)
    fit_sec = time.time() - t0

    y_pred = pipe.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    f1m = f1_score(y_val, y_pred, average="macro")

    scores = get_scores_for_auc(pipe, X_val)
    auc = roc_auc_score(y_val, scores) if (scores is not None and is_binary(y_val)) else np.nan

    print(f"[{name}] acc={acc:.4f} | f1m={f1m:.4f} | auc={auc if not np.isnan(auc) else float('nan'):.4f} | time={fit_sec:.1f}s")
    return {"name": name, "acc": acc, "f1m": f1m, "auc": auc, "time": fit_sec}


# PyTorch MLP 연습

# 🧩 희소 행렬(Sparse Matrix) vs 밀집 배열 (Dense Array)
Sparse Matrix: 대부분의 값이 0인 행렬. 메모리 절약을 위해 0이 아닌 값만 저장함.\
Dense Array: 모든 값을 다 저장하는 일반적인 배열

In [ ]:
from sklearn.pipeline import Pipeline as Skpipe
pre_only = Skpipe([("pre", pre)])
Xtr_t = pre_only.fit_transform(X_train)
# 데이터를 보고 학습(fit) + 변환(transform)을 동시에 수행
Xva_t = pre_only.transform(X_val)
# 이미 학습된 전처리기를 사용해서 변환만 수행

# 희소 행렬(sparse matrix) -> 밀집 배열(dense array)로 변환 
try:
    import scipy.sparse as sp
    if sp.issparse(Xtr_t): Xtr_t = Xtr_t.toarray()
    if sp.issparse(Xva_t): Xva_t = Xva_t.toarray()
except Exception:
    Xtr_t = np.asarray(Xtr_t); Xva_t = np.asarray(Xva_t)

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
# TensorDataset: 여러 개의 tensor들을 하나의 데이터셋 객체로 묶어줌 _ - 예: 입력 데이터와 정답(label)을 한 쌍으로 만들어서 모델이 학습할 수 있게 함.
# DataLoader: TensorDataset같은 데이터셋을 batch 단위로 나눠서 모델에 공급해주는 역할 
# -> 학습 시 미니배치 학습, 셔플, 병렬 처리 등을 쉽게 할 수 있음 
device = "cuda" if torch.cuda.is_available() else "cpu"

Xtr_tensor = torch.tensor(Xtr_t, dtype=torch.float32)
Xva_tensor = torch.tensor(Xva_t, dtype=torch.float32)
ytr_tensor = torch.tensor(np.array(y_train), dtype=torch.long)
yva_tensor = torch.tensor(np.array(y_val), dtype=torch.long)

train_loader = DataLoader(TensorDataset(Xtr_tensor, ytr_tensor), batch_size=256, shuffle=True)
val_loader = DataLoader(TensorDataset(Xva_tensor, yva_tensor), batch_size=512, shuffle=False)

in_dim = Xtr_t.shape[1]
model = nn.Sequential(
    nn.Linear(in_dim, 256), nn.ReLU(), nn.Dropout(0.15),
    nn.Linear(256, 128), nn.ReLU(), nn.Dropout(0.15),
    nn.Linear(128, 1)
).to(device)

# 불균형 대응: pos_weight = neg/pos
pos = (y_train==1).sum()
neg = (y_train==0).sum()
pos_weight = torch.tensor([neg / max(pos, 1)], dtype=torch.float, device=device)
# 혹시나 pos가 0일 경우 나눗셈 오류 방지 
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# Binary Cross Entropy Loss + Sigmoid를 합친 손실 함수 _ 이진 분류에서 자주 사용됨 
# sigmoid를 먼저 적용한 뒤 binary cross entropy 계산 
# pos weight -> for 불균형 해결 
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
# AdamW는 Adam optimizer의 변형이고, Weight Decay를 더 제대로 처리해주는 버전 
# 일반 Adam은 L2 정규화를 잘못 적용하는 문제가 있었는데 AdamW는 그걸 고침 
# model.parameters(): 모델의 학습 가능한 파라미터들을 가져옴 (가중치, 편향 등)
# lr=1e-3: 학습률을 0.001로 설정. 이 값은 파라미터를 얼마나 빠르게 업데이트할지를 결정
# AdamW: parameter update 방식으로 사용 _ 내부적으로는 momentum, adaptice learning rate, weight decay를 잘 조합함 
# Adam: (특징) 빠른 수렴, 적응적 학습률 (장점) 일반적인 상황에서 잘 작동
# AdamW: (특징) Weight Decay 분리 적용, (장점) 정규화가 필요한 모델에 더 안정적 

best_auc = -1.0
best_state = None
for epoch in range(1,21): 
    model.train()
    # 모델을 학습 모드로 설정 
    epoch_loss = 0.0
    # 한 epoch 동안의 총 손실을 저장할 변수 
    for xb, yb in train_loader:
        # train_loader는 DataLoader로부터 batch 단위로 데이터를 가져옴 
        xb, yb = xb.to(device), yb.to(device)
        # xb: 입력 데이터, yb: 정답 라벨 
        optimizer.zero_grad()
        # 이전 배치에서 계산된 graedient 초기화 
        logits = model(xb).squeeze(1)
        # 모델에 입력을 넣고 예측값(logits)을 얻음
        # squeeze(1)은 [batch_size, 1] -> [batch_size]로 차원 축소
        loss = criterion(logits, yb.float())
        # 예측값과 실제값을 비교해서 손실 계산 
        loss.backward()
        # 손실을 기준으로 gradient 계산 (역전파)
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # gradient 폭주 방지용. gradient의 norm을 최대 1.0으로 제한 
        optimizer.step()
        # 계산된 gradient 기반으로 parameter update 
        epoch_loss += loss.item() * xb.size(0)
        # loss.item: 현재 배치의 평균 손실 
        # 배치 평균 손실 × 배치 크기 = 배치 전체 손실 합
        # 현재 배치의 손실 값을 float로 가져옴
        # 이걸 epoch_loss에 계속 더해서 → 한 epoch 전체 손실 합을 구하는 거야
    epoch_loss /= len(train_loader.dataset)
    # 전체 손실을 샘플 수 기준으로 누적해서 평균 손실 계산 

    # val
    model.eval()
    probs, ys = [], []
    with torch.no_grad():
        # gradient 게산을 끔 -> 메모리 절약 + 속도 향상 
        # validation은 학습이 아니니까 gradient 필요 없음 
        for xb, yb in val_loader: # validation 데이터를 batch 단위로 불러옴 
            xb = xb.to(deivice)
            p = torch.sigmoid(model(xb).squeeze(1)).cpu().numpy()
            # 모델에 입력 넣고 logits 출력, 출력이 [batch_size, 1]일 경우 → squeeze(1)로 [batch_size]로 바꿔줌
            # logits를 확률로 변환 
            # sigmoi는 0~1 사이의 갑승로 바꿔주니까 이게 양성 클래스일 확률로 나옴
            probs.append(p); ys.append(yb.numpy())
            # p는 numpy 배열로 변환된 확률값 
            # 이걸 리스트에 계속 누적해서 전체 validation 결과 저장  
        probs = np.concatenate(probs); ys = np.concatenate(ys)
        preds = (probs >= 0.5).astype(int)

        f1m = f1_score(ys, preds, average="macro")
        try: 
            auc = roc_auc_score(ys, probs)
        except ValueError: 
            auc = np.nan

        if (auc if not np.isnan(auc) else -1) > best_auc: 
            # AUC가 이전보다 더 좋으면 모델 상태 저장 
            best_auc = auc 
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            # 모델의 모든 파라미터(가중치, 편향 등)를 딕셔너리 형태로 반환 
            # GPU에 있던 tensor를 CPU로 옮기고 복사함 _ 왜 복사하냐면 이후 학습이 계속되면 모델 파라미터가 바뀌니까, 그 순간의 상태를 안전하게 저장하려고  
        
        print(f"[EP {epoch:02d}] train_loss={epoch_loss:.4f} val_f1={f1m:.4f} val_auc={auc:.4f}")

if best_state is not None:
    model.load_state_dict(best_state)
print(f"Best DL AUC: {best_auc:.4f}")

# 데이터 형태 정확하게 맞춰주기

PyTorch에서는 tensor와 numpy array 사이의 변환이 자주 일어남 _ 정리 필요 

### 🔁 tensor vs numpy array 차이점
| 항목            | torch.Tensor                  | numpy.ndarray                  |
|-----------------|-------------------------------|--------------------------------|
| 사용 목적       | PyTorch 모델 학습, 연산       | 일반적인 수치 계산, 평가        |
| GPU 사용 가능   | ✅ 가능 (`.to(device)`)       | ❌ 불가능                       |
| 자동 미분       | ✅ 가능 (`.backward()`)       | ❌ 불가능                       |
| 모델 입력       | ✅ 사용됨                     | ❌ 직접 입력 불가               |
| 평가 지표 계산  | ❌ 불편함                     | ✅ `sklearn` 등에서 사용 용이   |


### 📥 모델에 들어갈 때: torch.Tensor
- 모델에 입력할 때는 무조건 tensor 형태여야 해
- 예: model(xb)에서 xb는 tensor여야 함
- GPU에서 학습하려면 .to(device)도 꼭 해줘야 함\
>xb = xb.to(device)  # tensor 형태로 GPU에 올림\
logits = model(xb)  # 모델에 입력



### 📤 모델에서 나올 때: tensor → numpy로 변환
- 모델 출력은 tensor 형태야
- 평가할 때는 numpy로 바꿔서 sklearn 같은 라이브러리에 넘겨줘야 함
>p = torch.sigmoid(logits).cpu().numpy()  # 확률로 바꾸고 numpy로 변환


- .cpu()는 GPU에서 CPU로 옮기는 작업
- .numpy()는 tensor → numpy array로 변환

### 🎯 왜 변환하냐면...
- PyTorch는 학습에 최적화된 프레임워크
- numpy는 평가 지표 계산, 시각화 등에 더 적합
- 예: roc_auc_score, confusion_matrix, plot() 등은 numpy를 요구함

### 💡 흐름 요약
- 데이터 로딩: numpy → tensor로 변환 (TensorDataset)
- 모델 학습: tensor 형태로 GPU에 올려서 학습
- 모델 출력: tensor 형태로 나옴
- 평가 단계: tensor → numpy로 변환해서 지표 계산


# 머신러닝 데이터 형태 정의 

대부분의 머신러닝 library(ex: scikit-learn)은 numpy.ndarray 또는 pandas.DataFrame 형태의 데이터 사용함 

# numpy.ndarray vs torch.Tensor
> np_arr = np.array([[1, 2], [3, 4]])       # shape: (2, 2)\
tensor = torch.tensor([[1, 2], [3, 4]])   # shape: (2, 2)

> **numpy 연산**\
>np_arr = np.array([[1, 2], [3, 4]])
>print(np_arr * 2)  # 단순 곱셈

> **tensor 연산**\
>tensor = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32, requires_grad=True)
print(tensor * 2)  # 곱셈 + gradient 추적 가능
